In [ ]:

!pip install mediapipe==0.10.13


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from base64 import b64decode

print("MediaPipe version:", mp.__version__)


In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)


In [ ]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)


In [ ]:
def take_photo():
    js = Javascript('''
        async function takePhoto() {
          const div = document.createElement('div');
          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});
          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          await new Promise(resolve => setTimeout(resolve, 500));
          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', 0.8);
        }
        takePhoto();
    ''')
    display(js)
    data = input("Press Enter after webcam image is captured")
    image_data = data.split(',')[1]
    img = cv2.imdecode(np.frombuffer(b64decode(image_data), np.uint8), cv2.IMREAD_COLOR)
    return img


In [ ]:
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
EYE_AR_THRESH = 0.25

img = take_photo()
h, w = img.shape[:2]

rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = face_mesh.process(rgb)

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        left_eye, right_eye = [], []

        for idx in LEFT_EYE:
            x = int(face_landmarks.landmark[idx].x * w)
            y = int(face_landmarks.landmark[idx].y * h)
            left_eye.append((x, y))
            cv2.circle(img, (x, y), 2, (0, 255, 0), -1)

        for idx in RIGHT_EYE:
            x = int(face_landmarks.landmark[idx].x * w)
            y = int(face_landmarks.landmark[idx].y * h)
            right_eye.append((x, y))
            cv2.circle(img, (x, y), 2, (0, 255, 0), -1)

        ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0

        if ear < EYE_AR_THRESH:
            cv2.putText(img, "DROWSINESS DETECTED", (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        else:
            cv2.putText(img, "AWAKE", (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

        cv2.putText(img, f"EAR: {ear:.2f}", (30, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

cv2_imshow(img)


<IPython.core.display.Javascript object>